# The N-Queens Problem

The N-Queens Problems is to place N queens on an N x N chessboard such that no queen attacks any other queen. A queen can attack any other queen if they are in the same row, column, or diagonal. The following figure shows a solution to the 8-queens problem.

<img src="img/nqueens.jpg" alt="nqueens" style="width:600px; height:400px;">

In this notebook, we will show how to model and solve the n-queens problem using SeaPearl.jl.

## Setup
We will begin by activating the environment and importing the necessary packages.

In [2]:
using Pkg
Pkg.activate("../../../../")
Pkg.instantiate()
using Revise
using SeaPearl

  Activating project at `c:\Users\leobo\Desktop\École\Poly\SeaPearl\SeaPearlZoo.jl`


## Modeling the Problem

We will now build a model for the problem. The first step is to create a model, implemented in SeaPearl by the `CPModel` struct. `CPModel` needs a trailer to keep track of its current position during search. Therefore, creating a model can be done in the following way:
```julia
trailer = SeaPearl.Trailer()
model = SeaPearl.CPModel(trailer)
```

Next up, we will be using integer variables, implemented as `SeaPearl.IntVar` structs. Creating such variables can be done in the following way:

```julia
SeaPearl.IntVar(minimum_value, maximum_value, variable_name, trailer)
```

We will proceed by creating an array for these variables, one for each vertex in the graph. Then, once variables are created, they need to be tied with a `CPModel` by the way of constraints. For example, to add an Equality constraint ensuring the variable `x` is equal to `1`, we can do the following:

```julia
push!(model.constraints, SeaPearl.EqualConstant(x, 1, trailer))
```

Finally, the model needs an objective in order to optimize. Keep in mind that SeaPearl always minimizes, although this should not be a problem for graph coloring. To add an objective, we can do the following:

```julia
SeaPearl.addVariable!(model, x)
SeaPearl.addObjective!(model, x)
```

Putting it all together, we have the following model.

In [ ]:
board_size = 8
trailer = SeaPearl.Trailer()
model = SeaPearl.CPModel(trailer)

# rows[i] designates the row of queen in column i
rows = Vector{SeaPearl.AbstractIntVar}(undef, board_size)
for i = 1:board_size
    rows[i] = SeaPearl.IntVar(1, board_size, "row_" * string(i), trailer)
    SeaPearl.addVariable!(model, rows[i]; branchable=true)
end
# diagonals from top left to bottom right
rows_plus = Vector{SeaPearl.AbstractIntVar}(undef, board_size)
for i = 1:board_size
    rows_plus[i] = SeaPearl.IntVarViewOffset(rows[i], i, rows[i].id * "+" * string(i))
end
# diagonals top right to bottom left
rows_minus = Vector{SeaPearl.AbstractIntVar}(undef, board_size)
for i = 1:board_size
    rows_minus[i] = SeaPearl.IntVarViewOffset(rows[i], -i, rows[i].id * "-" * string(i))
end

push!(model.constraints, SeaPearl.AllDifferent(rows, trailer)) # All rows and columns are different - since rows are all different and queens are on different rows
push!(model.constraints, SeaPearl.AllDifferent(rows_plus, trailer))
push!(model.constraints, SeaPearl.AllDifferent(rows_minus, trailer))

## Solving the Problem

The model is built; let's solve it!

In [4]:
valueSelection = SeaPearl.BasicHeuristic()
SeaPearl.solve!(model; variableHeuristic=SeaPearl.MinDomainVariableSelection{false}(), valueSelection=valueSelection)

:Optimal

## Visualizing the Solution

Let's visualize the solution obtained

In [5]:
"""
    print_queens(model::SeaPearl.CPModel; nb_sols=typemax(Int))

Print at max nb_sols solutions to the N-queens problems.

# Arguments
- `model::SeaPearl.CPModel`: needs the model to be already solved (by solve_queens)
- 'nb_sols::Int' : maximum number of solutions to print
"""
function print_queens(model::SeaPearl.CPModel; nb_sols=typemax(Int))
    variables = model.variables
    solutions = model.statistics.solutions
    board_size = length(model.variables)
    count = 0
    real_solutions = filter(e -> !isnothing(e), solutions)
    println("The solver found " * string(length(real_solutions)) * " solutions to the " * string(board_size) * "-queens problem. Let's show them.")
    println()
    for key in keys(real_solutions)
        if (count >= nb_sols)
            break
        end
        sol = real_solutions[key]
        println("Solution " * string(count + 1))
        count += 1
        for i in 1:board_size
            ind_queen = sol["row_"*string(i)]
            for j in 1:board_size
                if (j == ind_queen)
                    print("Q ")
                else
                    print("_ ")
                end
            end
            println()
        end
        println()
    end
end

print_queens

In [ ]:
print_queens(model)

## Appendix - Variable Selection Heuristic

SeaPearl allows for the creation of custom value selection and variable selection heuristics. For this example, we will build a custom variable selection heuristic.

The `MostCenteredVariableSelection` strategy selects the next variable to branch on based on how close the corresponding row is to the center of the board. The closer the row is to the center, the higher its score. The score is calculated using the `get_centered_score` function, which takes the number of variables (i.e., the size of the board) and a branchable variable (i.e., a variable that has not yet been assigned a value) as input and returns a floating-point score.

The `MostCenteredVariableSelection` strategy has two methods: one for selecting the next variable to branch on when the objective function is not taken into account (`(::MostCenteredVariableSelection{false})(cpmodel::SeaPearl.CPModel)::SeaPearl.AbstractIntVar`) and one for selecting the next variable to branch on when the objective function is taken into account (`(::MostCenteredVariableSelection{true})(cpmodel::SeaPearl.CPModel)::SeaPearl.AbstractIntVar`). 

Both methods take a `SeaPearl.CPModel` object as input and return a `SeaPearl.AbstractIntVar` object, which represents the next variable to branch on. The methods first collect all branchable variables in the model, then sort them based on their centered score using the `get_centered_score` function. Finally, the methods loop over the sorted variables until an unbound variable is found, and return that variable.

The `get_centered_score` function takes the number of variables and a branchable variable as input and returns the centered score of the corresponding row. The centered score is calculated as the absolute difference between the row index and the center of the board.

In [9]:
"""
    struct MostCenteredVariableSelection{TakeObjective}

VariableSelection heuristic that selects the legal (ie. among the not bounded ones) most centered Queen.
"""
struct MostCenteredVariableSelection{TakeObjective} <: SeaPearl.AbstractVariableSelection{TakeObjective} end
MostCenteredVariableSelection(; take_objective=true) = MostCenteredVariableSelection{take_objective}()

function (::MostCenteredVariableSelection{false})(cpmodel::SeaPearl.CPModel)::SeaPearl.AbstractIntVar
    selected_variable = nothing
    num_variables = length(cpmodel.variables)
    branchable_variables = collect(SeaPearl.branchable_variables(cpmodel))

    # sorted_variables will be of type Vector{Pair{String, SeaPearl.AbstractVar}}
    # all elements of the sorted_variables Vector will contain the variable name in position 1
    # and the variable in position 2
    sorted_variables = sort(
        branchable_variables,
        by=x -> get_centered_score(num_variables, x),
        rev=true
    )
    # Loop until an unbound variable is found
    while !isempty(sorted_variables)
        selected_variable = pop!(sorted_variables)[2] # as mentionned above, the second element is the variable
        if !(selected_variable == cpmodel.objective) && !SeaPearl.isbound(selected_variable)
            break
        end
    end

    if SeaPearl.isnothing(selected_variable) && !SeaPearl.isbound(cpmodel.objective)
        return cpmodel.objective
    end
    return selected_variable
end

function (::MostCenteredVariableSelection{true})(cpmodel::SeaPearl.CPModel)::SeaPearl.AbstractIntVar # question: argument{true} ou {false} ?
    selected_variable = nothing
    num_variables = length(cpmodel.variables)
    branchable_variables = collect(SeaPearl.branchable_variables(cpmodel))
    sorted_variables = sort(
        branchable_variables,
        by=x -> get_centered_score(num_variables, x),
        rev=true
    )
    # Loop until an unbound variable is found
    while true
        selected_variable = pop!(sorted_variables)[2]
        if !SeaPearl.isbound(selected_variable)
            break
        end
    end

    return selected_variable
end
"""
    get_centered_score(num_variables::Int, branchable_variable::Pair{String, SeaPearl.AbstractVar})::Float64
Returns the centered score of a row; i.e. how close the queen is to the center of the board.
"""
function get_centered_score(num_variables::Int, branchable_variable::Pair{String,SeaPearl.AbstractVar})::Float64
    variable_name::String = branchable_variable[1]
    row_id::Int = parse(Int, match(r"[0-9]*$", variable_name).match)
    centered_score::Float64 = abs(num_variables / 2 - row_id)

    return centered_score
end

get_centered_score

## Solving the problem - with a custom variable selection heuristic

Now let's put this heuristic to use!

In [ ]:
valueSelection = SeaPearl.BasicHeuristic()
SeaPearl.solve!(model; variableHeuristic=MostCenteredVariableSelection{false}(), valueSelection=valueSelection)